In [5]:
from langchain_ollama import OllamaLLM
import langchain
import numpy
import pandas

#initialize model
lafilamod= OllamaLLM(
    model="Llama3:latest",
    temperature=0,
    num_ctx=2048,
    verbose=True,
)

In [6]:
from PyPDF2 import PdfReader

# Read pdf, get text
reader = PdfReader('/Users/abhimanyu/Downloads/Hands-On_Large_Language_Models_compressed.pdf')
page_ids = []
page_texts = []
for i, p in enumerate(reader.pages):
    page_ids.append(str(i))
    page_texts.append(p.extract_text())



In [7]:
print(page_ids)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '15

In [8]:
# from langchain.text_splitter import MarkdownTextSplitter

# # Create a MarkdownTextSplitter for semantic chunking
# semantic_splitter = MarkdownTextSplitter(
#     chunk_size=200,    # Number of characters per chunk
#     chunk_overlap=50   # Overlap to preserve context
# )

# # Split the text into semantic chunks
# semantic_chunks = semantic_splitter.split_text(page_texts)
# print("\nSemantic Chunks (Markdown-based):\n", semantic_chunks)


In [9]:
# #now embeddings for the text
# from sentence_transformers import SentenceTransformer

# embed_model=SentenceTransformer("")

# text_embeddings=embed_model.encode(page_texts)

# index_embeddings=embed_model.encode(page_ids)
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
embed_model=HuggingFaceBgeEmbeddings(model_name='thenlper/gte-small')

from langchain.vectorstores import FAISS
metadata= [{"page":int(i)} for i in page_ids]
database= FAISS.from_texts(page_texts, embed_model, metadatas=metadata)


/var/folders/ql/hdbyk3lx6vq0x6fs9227fc_m0000gn/T/ipykernel_50628/1651442095.py:10: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model=HuggingFaceBgeEmbeddings(model_name='thenlper/gte-small')
/Users/abhimanyu/Desktop/geospacy/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0804 09:57:47.185000 50628 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
/Users/abhimanyu/Desktop/geospacy/

In [12]:
from langchain.prompts import PromptTemplate

document_prompt = PromptTemplate(
    input_variables=["page_content"],
     template="{page_content}"
)

In [18]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import RegexParser
parser= RegexParser(
    regex=r"answer: (.*?)\nScore:(.*)",
    output_keys=["answer","score"],
)
# template1="""
# <|begin_of_text|><|start_header_id|>system<|end_header_id|>
# you are a search retrieval agent that answers precisely to the query.the format needs to be like:
# Answer: <some answer>
# Score: <number between 0 and 1>
# <|eot_id|>
# <|start_header_id|>user<|end_header_id|>
# provide a relevant answer according to the user query:
# {context}
# the format needs to be like:
# Answer: <some answer>
# Score: <number between 0 and 1>

# <|eod_id|>
# <|start_header_id|>assistant<|end_header_id|>"""

from langchain.prompts import PromptTemplate

# rag_prompt = PromptTemplate.from_template("""
# Given the following document and a question, provide a helpful answer and a score between 0 and 1 indicating how well the document answers the question.

# Document:
# {context}

# Question:
# {question}

# Answer the question and rate the relevance.

# Output format:
# Answer: <your answer>
# Score: <score between 0 and 1>
# """)
template1map="""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an intelligent assistant helping to answer a user's question using a provided document.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
provide a relevant answer according to the user query:
{question}.

Document: {summaries}

the format needs to be like:
Answer: <some answer>
Score: <number between 0 and 1>

<|eod_id|>
<|start_header_id|>assistant<|end_header_id|>"""

template1combine="""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a smart assistant combining multiple partial answers from different documents to answer a user’s question.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
question:{question}
partial answers: {summaries}

based on the above construct a concrete final answer for the user:
<|eod_id|>
<|start_header_id|>assistant<|end_header_id|>"""

map_prompt= PromptTemplate(template=template1map, input_variables=[ "question","summaries"])
combine_prompt= PromptTemplate(template=template1combine, input_variables=["question", "summaries"])
document_variable_name="summaries"
# from langchain.chains import RetrievalQA
# rag= RetrievalQA.from_chain_type(llm=lafilamod, chain_type="map_rerank", retriever= database.as_retriever(), chain_type_kwargs={'prompt': rag_prompt},output_parser=parser, verbose=True,)

In [22]:
from langchain.chains import RetrievalQA, LLMChain, MapReduceDocumentsChain, StuffDocumentsChain, ReduceDocumentsChain

# rag= RetrievalQA.from_chain_type(llm=lafilamod, chain_type="map_reduce", retriever= database.as_retriever(),chain_type_kwargs={'map_prompt': map_prompt,"combine_prompt": combine_prompt})

llm_chain_map= LLMChain(llm=lafilamod, prompt= map_prompt)
llm_chain_combine=LLMChain(llm=lafilamod, prompt=combine_prompt)

combine_docschain=StuffDocumentsChain(llm_chain=llm_chain_combine,     document_prompt=document_prompt,
document_variable_name=document_variable_name)

reduce_doc_chain=ReduceDocumentsChain(combine_documents_chain=combine_docschain)

map_reduce_chain= MapReduceDocumentsChain(
    llm_chain= llm_chain_map,
    reduce_documents_chain= reduce_doc_chain,
    document_variable_name=document_variable_name
)
from langchain.chains import RetrievalQA

rag= RetrievalQA( retriever= database.as_retriever(), combine_documents_chain= map_reduce_chain )

In [20]:
results = database.similarity_search("where is that part where langchain is used to agent of llm", k=1)
for doc in results:
    page = doc.metadata.get("page")
    print(f"Page: {page}\nContent: {doc.page_content}\n")


Page: 283
Content: llm.invoke("Hi! My name is Maarten. What is 1 + 1?" )
''
Unfortunately , we get no output! As we have seen in previous chapters, Phi-
3 requires a specific prompt template. Compared to our examples with
transformers , we will need to explicitly use a template ourselves.
Instead of copy-pasting this template each time we use Phi-3  in LangChain,
we can use one of LangChain’ s core functionalities, namely “chains.”
TIP
All examples in this chapter can be run with any LLM. This means that you can choose
whether to use Phi-3, ChatGPT , Llama 3 or anything else when going through the
examples. We will use Phi-3 as a default throughout, but the state-of-the-art changes
quickly , so consider using a newer model instead. You can use  the Open LLM
Leaderboard  (a ranking of open source LLMs) to choose whichever works best for your
use case.
If you do not have access to a device that can run LLMs locally , consider using
ChatGPT  instead:
from langchain .chat_models  import Ch

In [21]:
rag_response=rag.invoke("what is an llm agent?")
print(rag_response)


{'query': 'what is an llm agent?', 'result': 'Based on the provided partial answers, I can construct a concrete final answer for the user:\n\nAn LLM Agent is a type of artificial intelligence that combines the capabilities of a Large Language Model (LLM) with the ability to use various tools or APIs to solve complex problems. It interacts with the outside world using these tools, allowing it to perform specific tasks such as searching and calculating. To create an LLM agent, you need to load these tools and pass them to a ReAct (Reasoning and Acting) agent, which is responsible for executing the steps defined by the tools.'}


In [ ]:
template1="""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
you are a search retrieval agent that answers precisely to the query.the format needs to be like:
Answer: <some answer>
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
provide a relevant answer according to the user query:
{context}

<|eod_id|>
<|start_header_id|>assistant<|end_header_id|>"""

stuff_prompt=PromptTemplate(template= template1, input_variables=["context"])
ragstuff= RetrievalQA.from_chain_type(llm=lafilamod, chain_type="stuff", retriever= database.as_retriever(), chain_type_kwargs={'prompt': stuff_prompt}, verbose=True,)

In [29]:
def q(question):
    response= ragstuff.invoke(question)
    return response


In [30]:
q("what is an llm agent")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'what is an llm agent',
 'result': 'Answer: Memory type Pros Cons\n\nScore: 0.8'}